In [7]:
import numpy as np
import pandas as pd

In [42]:
# initial DB of test samples including nuclide vectors
sfco29_pkl = '~/sfcompo/format_clean/sfcompo_nuc29.pkl'
sfcoDB = pd.read_pickle(sfco29_pkl)

# results DBs with LogLikelihoods and predictions
sfco_path = '~/sims_n_results/simupdates_aug2020/sfco29/'
uncs = [0.05, 0.1, 0.15, 0.2]
job_dirs = ['Job' + str(i) + '_unc' + str(unc) for i, unc in enumerate(uncs)]
sfcoLL = {}
for i, unc_job in enumerate(job_dirs):
    sfcoLL[str(i)] = pd.read_csv(sfco_path + unc_job + '/' + unc_job + '.csv').drop(columns=['Unnamed: 0', 'Unnamed: 0.1'])
    
lbls = ['ReactorType', 'CoolingTime', 'Enrichment', 'Burnup', 'OrigenReactor']

In [43]:
def score(error, y_true):
    zero_cnt = len(y_true) - np.count_nonzero(y_true)
    if zero_cnt > 0:
        error, y_true = zip(*((e, y) for e, y in zip(error, y_true) if y != 0))
        error = np.array(error)
        y_true = np.array(y_true)
    return 1 - np.mean(error / y_true)

In [44]:
for unc_df in list(sfcoLL.keys()):
    df = sfcoLL[unc_df]
    df['Rel_Burn_Error'] = df['Burnup_Error'] / df['Burnup']
    df['Rel_Enri_Error'] = df['Enrichment_Error'] / df['Enrichment']
df = df.replace([np.inf, -np.inf], np.nan)

In [45]:
sfcoLL['0'].columns

Index(['sim_idx', 'ReactorType', 'CoolingTime', 'Enrichment', 'Burnup',
       'OrigenReactor', 'pred_idx', 'pred_ReactorType', 'pred_CoolingTime',
       'pred_Enrichment', 'pred_Burnup', 'pred_OrigenReactor',
       'AvgPowerDensity', 'ModDensity', 'UiWeight', 'MaxLogLL', 'MaxLLUnc',
       '2ndMaxLogLL', '2ndMaxLLUnc', 'CDF_LogLL_0.9998', 'CDF_LLUnc_0.9998',
       'CDF_LogLL_0.9988', 'CDF_LLUnc_0.9988', 'CDF_LogLL_0.95',
       'CDF_LLUnc_0.95', 'CDF_LogLL_0.9', 'CDF_LLUnc_0.9', 'CDF_LogLL_0.5',
       'CDF_LLUnc_0.5', 'CDF_LogLL_0.1', 'CDF_LLUnc_0.1', 'CDF_LogLL_0.01',
       'CDF_LLUnc_0.01', 'ReactorType_Score', 'CoolingTime_Error',
       'Enrichment_Error', 'Burnup_Error', 'OrigenReactor_Score',
       'Rel_Burn_Error', 'Rel_Enri_Error'],
      dtype='object')

In [46]:
sfcoDB.columns

Index(['Burnup', 'CoolingTime', 'Enrichment', 'OrigenReactor', 'ReactorType',
       'am241', 'am242m', 'am243', 'cm242', 'cm244', 'cs134', 'cs137', 'eu154',
       'nd143', 'nd144', 'nd145', 'nd146', 'nd148', 'nd150', 'np237', 'pu238',
       'pu239', 'pu240', 'pu241', 'pu242', 'sm147', 'sm149', 'sm150', 'sm151',
       'sm152', 'u234', 'u235', 'u236', 'u238'],
      dtype='object')

In [47]:
to_print = ['sim_idx', 'pred_idx', 
            'ReactorType', 'pred_ReactorType', 'ReactorType_Score', 
            'Enrichment', 'pred_Enrichment', 'Rel_Enri_Error', 
            'Burnup', 'pred_Burnup', 'Rel_Burn_Error', 
            'pred_CoolingTime']

In [48]:
# low burnup error cases
low_error = sfcoLL['0'].loc[sfcoLL['0']['Rel_Burn_Error'] < 0.005, to_print]
low_error

,sim_idx,pred_idx,ReactorType,pred_ReactorType,ReactorType_Score,Enrichment,pred_Enrichment,Rel_Enri_Error,Burnup,pred_Burnup,Rel_Burn_Error,pred_CoolingTime
7,CCL-1|D047|MKP109|LL,344552,pwr,pwr,True,3.038,3.040,0.000658,27350.0,27280.53,0.002540,3031.473636
26,FDN-1|2F1ZN3|C3|UT,287134,bwr,pwr,False,4.900,4.520,0.077551,59050.0,58841.98,0.003523,3475.938885
86,GRM-1|C16|E5|K2680,37743,bwr,bwr,True,2.530,2.740,0.083004,19240.0,19250.15,0.000528,6.392266
102,HBR-2|BO-5|N9|B-S,30878,pwr,bwr,False,2.561,2.740,0.069895,16020.0,16053.83,0.002112,3590.047747
105,HBR-2|BO-5|N9|C-J,319361,pwr,pwr,True,2.561,2.170,0.152675,28470.0,28470.89,0.000031,4176.997564
117,JPD-1|A-18|C3|+049,211873,bwr,bwr,True,2.600,2.730,0.050000,4304.0,4283.63,0.004733,967.271577
217,NOV-4|13602496|91|24,305705,pwr,pwr,True,3.600,3.640,0.011111,39700.0,39669.85,0.000759,70.398920
247,NPD-1|1096|A|1,361157,phwr,phwr,True,0.711,0.711,0.000000,5337.0,5315.32,0.004062,1539.710736
292,OBR-1|BE124|E3|P3,344705,pwr,pwr,True,3.000,3.040,0.013333,36200.0,36081.59,0.003271,104.267738
318,TAK-3|NT3G24|SF97|1,46084,pwr,bwr,False,4.110,4.400,0.070560,17690.0,17668.74,0.001202,25.487227


## Explore high-burnup error cases

In [49]:
hundo_error = sfcoLL['0'].loc[sfcoLL['0']['Rel_Burn_Error'] > 1.0, to_print]
hundo_error

,sim_idx,pred_idx,ReactorType,pred_ReactorType,ReactorType_Score,Enrichment,pred_Enrichment,Rel_Enri_Error,Burnup,pred_Burnup,Rel_Burn_Error,pred_CoolingTime
54,GAR-1|SA-13|E6|11,111254,bwr,bwr,True,2.41,5.23,1.170124,4200.0,8683.04,1.067390,1136.709042
63,GAR-1|A-106|E5|1,309124,bwr,pwr,False,2.10,5.48,1.609524,8930.0,21793.78,1.440513,29.215503
65,GAR-1|A-106|D2|1,173820,bwr,bwr,True,2.10,5.11,1.433333,9440.0,19593.53,1.075586,0.000602


In [40]:
# worst Burnup pred: sim_idx is GAR-1|A-106|E5|1

# 63 is 140% error:
#hundo_error.loc['GAR-1|A-106|E5|1']

In [41]:
sfcoLL['0'].loc[sfcoLL['0']['sim_idx'] == 'GAR-1|A-106|E5|1']

,sim_idx,ReactorType,CoolingTime,Enrichment,Burnup,OrigenReactor,pred_idx,pred_ReactorType,pred_CoolingTime,pred_Enrichment,...,CDF_LogLL_0.01,CDF_LLUnc_0.01,ReactorType_Score,CoolingTime_Error,Enrichment_Error,Burnup_Error,OrigenReactor_Score,Rel_Burn_Error,Rel_Enri_Error,Rel_Cool_Error
63,GAR-1|A-106|E5|1,bwr,0.0,2.1,8930.0,Garigliano-1_BWR,309124,pwr,29.215503,5.48,...,-4.724834e+18,43.689783,False,29.215503,3.38,12863.78,False,1.440513,1.609524,inf


## Explore how many ratio entries are 0

In [16]:
# do not rerun
ratios['> zero'] = ratios[ratios.columns[~ratios.columns.isin(lbls)]].gt(0).sum(axis=1)
ratios['== zero'] = ratios[ratios.columns[~ratios.columns.isin(lbls)]].eq(0).sum(axis=1)

In [17]:
ratios.head()

,ReactorType,CoolingTime,Enrichment,Burnup,OrigenReactor,cs137/cs133,cs134/cs137,cs135/cs137,ba136/ba138,sm150/sm149,sm152/sm149,eu154/eu153,pu240/pu239,pu241/pu239,pu242/pu239,> zero,== zero
BAL-2|1476|42|15,pwr,0.0,4.4,45100.0,Balakovo-2_VVER-1000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.425041,0.280066,0.113674,3,7
BAL-2|1476|42|31,pwr,0.0,4.4,45800.0,Balakovo-2_VVER-1000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.428110,0.274637,0.113086,3,7
BAL-2|1476|42|6,pwr,0.0,4.4,46700.0,Balakovo-2_VVER-1000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.437705,0.288525,0.118033,3,7
BAL-3|1591|23|912,pwr,0.0,4.4,46200.0,Balakovo-3_VVER-1000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.418530,0.281150,0.119808,3,7
BAL-3|1591|23|581,pwr,0.0,4.4,47900.0,Balakovo-3_VVER-1000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.440789,0.289474,0.144737,3,7


In [18]:
count = ratios.loc[ratios['> zero'] == 9]
count.head()

,ReactorType,CoolingTime,Enrichment,Burnup,OrigenReactor,cs137/cs133,cs134/cs137,cs135/cs137,ba136/ba138,sm150/sm149,sm152/sm149,eu154/eu153,pu240/pu239,pu241/pu239,pu242/pu239,> zero,== zero
DOD-1|Y013|UO2-b2|DU1,bwr,0.0,4.94,55490.0,Dodewaard-1_BWR,1.000000,0.012397,0.338602,0.0,181.385814,66.051661,0.113984,0.684829,0.213328,0.213328,9,1
GOS-1|1701|16B05|GU4 (Cycle 18),pwr,0.0,4.10,29070.0,Gösgen-1_PWR,0.919501,0.041622,0.375113,0.0,80.450630,31.348575,0.147344,0.356064,0.191205,0.059996,9,1
GOS-1|1701|16B05|GU3 (Cycle 18),pwr,0.0,4.10,52500.0,Gösgen-1_PWR,1.074341,0.081027,0.268415,0.0,96.816114,28.265107,0.198184,0.489219,0.289633,0.175091,9,1
GOS-1|1701|16B05|GU3' (Cycle 18),pwr,0.0,4.10,52500.0,Gösgen-1_PWR,1.143944,0.068570,0.254712,0.0,132.757594,39.874926,0.188847,0.487285,0.278694,0.174055,9,1
GOS-1|1240 (Cycle 15)|14H13|GU1 (Cycle 15),pwr,0.0,3.50,59660.0,Gösgen-1_PWR,1.178074,0.053422,0.254161,0.0,154.986276,50.503202,0.153502,0.649622,0.294743,0.317856,9,1


In [19]:
ratios['> zero'].describe()

count    544.000000
mean       3.867647
std        1.722172
min        2.000000
25%        3.000000
50%        3.000000
75%        4.000000
max        9.000000
Name: > zero, dtype: float64

In [ ]:
ratios

## Explore 9 ratios cases

In [56]:
rat9 = ratios.loc[ratios['> zero'] == 9]
rat9.head()

,ReactorType,CoolingTime,Enrichment,Burnup,OrigenReactor,cs137/cs133,cs134/cs137,cs135/cs137,ba136/ba138,sm150/sm149,sm152/sm149,eu154/eu153,pu240/pu239,pu241/pu239,pu242/pu239,> zero,== zero
DOD-1|Y013|UO2-b2|DU1,bwr,0.0,4.94,55490.0,Dodewaard-1_BWR,1.000000,0.012397,0.338602,0.0,181.385814,66.051661,0.113984,0.684829,0.213328,0.213328,9,1
GOS-1|1701|16B05|GU4 (Cycle 18),pwr,0.0,4.10,29070.0,Gösgen-1_PWR,0.919501,0.041622,0.375113,0.0,80.450630,31.348575,0.147344,0.356064,0.191205,0.059996,9,1
GOS-1|1701|16B05|GU3 (Cycle 18),pwr,0.0,4.10,52500.0,Gösgen-1_PWR,1.074341,0.081027,0.268415,0.0,96.816114,28.265107,0.198184,0.489219,0.289633,0.175091,9,1
GOS-1|1701|16B05|GU3' (Cycle 18),pwr,0.0,4.10,52500.0,Gösgen-1_PWR,1.143944,0.068570,0.254712,0.0,132.757594,39.874926,0.188847,0.487285,0.278694,0.174055,9,1
GOS-1|1240 (Cycle 15)|14H13|GU1 (Cycle 15),pwr,0.0,3.50,59660.0,Gösgen-1_PWR,1.178074,0.053422,0.254161,0.0,154.986276,50.503202,0.153502,0.649622,0.294743,0.317856,9,1


In [21]:
rat9IsoLL = pd.merge(rat9, sfcoLL['0'], left_index=True, right_on='sim_idx')

In [22]:
rat9IsoLL.columns

Index(['ReactorType_x', 'CoolingTime_x', 'Enrichment_x', 'Burnup_x',
       'OrigenReactor_x', 'cs137/cs133', 'cs134/cs137', 'cs135/cs137',
       'ba136/ba138', 'sm150/sm149', 'sm152/sm149', 'eu154/eu153',
       'pu240/pu239', 'pu241/pu239', 'pu242/pu239', '> zero', '== zero',
       'Unnamed: 0', 'Unnamed: 0.1', 'sim_idx', 'ReactorType_y',
       'CoolingTime_y', 'Enrichment_y', 'Burnup_y', 'OrigenReactor_y',
       'pred_idx', 'pred_ReactorType', 'pred_CoolingTime', 'pred_Enrichment',
       'pred_Burnup', 'pred_OrigenReactor', 'MaxLogLL', 'MaxLLUnc',
       '2ndMaxLogLL', '2ndMaxLLUnc', 'CDF_LogLL_0.9998', 'CDF_LLUnc_0.9998',
       'CDF_LogLL_0.9988', 'CDF_LLUnc_0.9988', 'CDF_LogLL_0.95',
       'CDF_LLUnc_0.95', 'CDF_LogLL_0.9', 'CDF_LLUnc_0.9', 'CDF_LogLL_0.5',
       'CDF_LLUnc_0.5', 'CDF_LogLL_0.1', 'CDF_LLUnc_0.1', 'CDF_LogLL_0.01',
       'CDF_LLUnc_0.01', 'ReactorType_Score', 'CoolingTime_Error',
       'Enrichment_Error', 'Burnup_Error', 'OrigenReactor_Score',
       'Per

In [23]:
sfcoLL['0']['MaxLogLL'].describe()

count     544.000000
mean      -28.584146
std       321.242060
min     -7251.046687
25%       -20.345720
50%         9.934160
75%        12.193510
max        21.811218
Name: MaxLogLL, dtype: float64

In [24]:
rat9IsoLL['MaxLogLL'].describe()

count      39.000000
mean     -310.521263
std      1170.475640
min     -7251.046687
25%      -141.760022
50%       -83.842861
75%       -38.314910
max        -3.736192
Name: MaxLogLL, dtype: float64

## Explore highest LL predictions

Note the best predictions in terms of LL have burnup errors > 100% for PHWRs

In [25]:
to_print = ['sim_idx', 'ReactorType', 'Enrichment', 'Burnup', 'pred_idx', 'pred_ReactorType', 'pred_Enrichment', 'pred_Burnup', 'ReactorType_Score', 'Enrichment_Error', 'Burnup_Error', 'Percent_Burnup_Error', 'MaxLogLL']
bestLL = sfcoLL['0'].loc[sfcoLL['0']['MaxLogLL'] > 18, to_print]
bestLL

,sim_idx,ReactorType,Enrichment,Burnup,pred_idx,pred_ReactorType,pred_Enrichment,pred_Burnup,ReactorType_Score,Enrichment_Error,Burnup_Error,Percent_Burnup_Error,MaxLogLL
138,JPD-1|A-18|C3|-293,bwr,2.600,2907.0,21894,bwr,2.19,2934.73,True,0.410,27.73,0.009539,21.811218
139,JPD-1|A-14|C3|+538,bwr,2.600,3344.0,18006,bwr,4.35,3145.26,True,1.750,198.74,0.059432,20.445807
141,JPD-1|A-14|C3|-293,bwr,2.600,4085.0,6855,pwr,1.91,3987.23,False,0.690,97.77,0.023934,19.947932
143,JPD-1|A-18|C3|+049,bwr,2.600,4304.0,3615,pwr,2.15,4136.97,False,0.450,167.03,0.038808,19.901196
182,MIH-3|JPNNM3SFA1|E1|86B03,pwr,3.208,6900.0,23406,pwr,3.82,5670.58,True,0.612,1229.42,0.178177,20.833871
269,NPD-1|1022|C|1,phwr,0.711,1059.0,14605,bwr,2.16,2826.34,False,1.449,1767.34,1.668876,21.271682
270,NPD-1|1129|A|1,phwr,0.711,1223.0,18736,bwr,1.62,3244.15,False,0.909,2021.15,1.652617,21.770452
271,NPD-1|1129|B|1,phwr,0.711,1306.0,196,pwr,1.41,3609.51,False,0.699,2303.51,1.763790,21.222108
272,NPD-1|1129|C|1,phwr,0.711,1622.0,17287,bwr,1.47,4174.70,False,0.759,2552.70,1.573798,20.249597
273,NPD-1|1630|A|1,phwr,0.711,2733.0,17052,bwr,0.54,5915.89,False,0.171,3182.89,1.164614,18.430371


In [26]:
bestLL = sfcoLL['0'].loc[sfcoLL['0']['MaxLogLL'] > 18]
bestIsoLL = pd.merge(ratios, bestLL, left_index=True, right_on='sim_idx')

In [27]:
bestIsoLL.columns

Index(['ReactorType_x', 'CoolingTime_x', 'Enrichment_x', 'Burnup_x',
       'OrigenReactor_x', 'cs137/cs133', 'cs134/cs137', 'cs135/cs137',
       'ba136/ba138', 'sm150/sm149', 'sm152/sm149', 'eu154/eu153',
       'pu240/pu239', 'pu241/pu239', 'pu242/pu239', '> zero', '== zero',
       'Unnamed: 0', 'Unnamed: 0.1', 'sim_idx', 'ReactorType_y',
       'CoolingTime_y', 'Enrichment_y', 'Burnup_y', 'OrigenReactor_y',
       'pred_idx', 'pred_ReactorType', 'pred_CoolingTime', 'pred_Enrichment',
       'pred_Burnup', 'pred_OrigenReactor', 'MaxLogLL', 'MaxLLUnc',
       '2ndMaxLogLL', '2ndMaxLLUnc', 'CDF_LogLL_0.9998', 'CDF_LLUnc_0.9998',
       'CDF_LogLL_0.9988', 'CDF_LLUnc_0.9988', 'CDF_LogLL_0.95',
       'CDF_LLUnc_0.95', 'CDF_LogLL_0.9', 'CDF_LLUnc_0.9', 'CDF_LogLL_0.5',
       'CDF_LLUnc_0.5', 'CDF_LogLL_0.1', 'CDF_LLUnc_0.1', 'CDF_LogLL_0.01',
       'CDF_LLUnc_0.01', 'ReactorType_Score', 'CoolingTime_Error',
       'Enrichment_Error', 'Burnup_Error', 'OrigenReactor_Score',
       'Per

In [28]:
bestIsoLL[['> zero']]

,> zero
138,3
139,3
141,3
143,3
182,4
269,3
270,3
271,3
272,3
273,3


## Explore all available Isos in SFCOMPO, not just TAMU-filtered

In [4]:
allIsos = pd.read_pickle('~/sfcompo/format_clean/sfcompo_all_isos_formatted.pkl')
allIsos.head()

,Burnup,CoolingTime,Enrichment,OrigenReactor,ReactorType,ag109,ag110m,am,am241,am242,...,sm154,sr90,tc99,u,u232,u233,u234,u235,u236,u238
BAL-2|1476|42|15,45100.0,0.0,4.4,Balakovo-2_VVER-1000,pwr,0.0,0.0,0.0,0.048,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000,10.64,6.17,926.55
BAL-2|1476|42|31,45800.0,0.0,4.4,Balakovo-2_VVER-1000,pwr,0.0,0.0,0.0,0.000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.226,10.66,6.00,925.00
BAL-2|1476|42|6,46700.0,0.0,4.4,Balakovo-2_VVER-1000,pwr,0.0,0.0,0.0,0.066,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000,10.27,6.11,925.18
BAL-3|1591|23|912,46200.0,0.0,4.4,Balakovo-3_VVER-1000,pwr,0.0,0.0,0.0,0.054,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000,10.18,6.15,925.48
BAL-3|1591|23|581,47900.0,0.0,4.4,Balakovo-3_VVER-1000,pwr,0.0,0.0,0.0,0.071,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000,8.18,6.34,925.66


In [10]:
# do not rerun
allIsos['> zero'] = allIsos[allIsos.columns[~allIsos.columns.isin(lbls)]].gt(0).sum(axis=1)
allIsos['== zero'] = allIsos[allIsos.columns[~allIsos.columns.isin(lbls)]].eq(0).sum(axis=1)
allIsos

,Burnup,CoolingTime,Enrichment,OrigenReactor,ReactorType,ag109,ag110m,am,am241,am242,...,tc99,u,u232,u233,u234,u235,u236,u238,> zero,== zero
BAL-2|1476|42|15,45100.0,0.0,4.4,Balakovo-2_VVER-1000,pwr,0.0,0.0,0.0,0.048,0.0,...,0.0,0.0,0.0,0.0,0.000,10.640,6.170,926.55,17,68
BAL-2|1476|42|31,45800.0,0.0,4.4,Balakovo-2_VVER-1000,pwr,0.0,0.0,0.0,0.000,0.0,...,0.0,0.0,0.0,0.0,0.226,10.660,6.000,925.00,13,72
BAL-2|1476|42|6,46700.0,0.0,4.4,Balakovo-2_VVER-1000,pwr,0.0,0.0,0.0,0.066,0.0,...,0.0,0.0,0.0,0.0,0.000,10.270,6.110,925.18,17,68
BAL-3|1591|23|912,46200.0,0.0,4.4,Balakovo-3_VVER-1000,pwr,0.0,0.0,0.0,0.054,0.0,...,0.0,0.0,0.0,0.0,0.000,10.180,6.150,925.48,17,68
BAL-3|1591|23|581,47900.0,0.0,4.4,Balakovo-3_VVER-1000,pwr,0.0,0.0,0.0,0.071,0.0,...,0.0,0.0,0.0,0.0,0.000,8.180,6.340,925.66,17,68
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YNK-1|E6|NW-A1|G-127,42500.0,0.0,3.4,Yankee-1_PWR,pwr,0.0,0.0,0.0,0.000,0.0,...,0.0,0.0,0.0,0.0,0.111,7.249,4.770,932.06,10,75
YNK-1|E6|NW-A1|G-128,42500.0,0.0,3.4,Yankee-1_PWR,pwr,0.0,0.0,0.0,0.000,0.0,...,0.0,0.0,0.0,0.0,0.105,7.125,4.598,932.39,10,75
YNK-1|E6|NW-A1|N-21,42600.0,0.0,3.4,Yankee-1_PWR,pwr,0.0,0.0,0.0,0.000,0.0,...,0.0,0.0,0.0,0.0,0.102,7.373,4.569,931.37,8,77
YNK-1|E6|NW-A1|T-165,43200.0,0.0,3.4,Yankee-1_PWR,pwr,0.0,0.0,0.0,0.000,0.0,...,0.0,0.0,0.0,0.0,0.104,7.020,4.703,931.16,9,76


In [11]:
allIsos['> zero'].describe()

count    544.000000
mean      17.608456
std       11.805795
min        5.000000
25%        9.000000
50%       12.000000
75%       26.000000
max       66.000000
Name: > zero, dtype: float64

In [31]:
#df = allIsos.loc[allIsos['> zero']<8]
#df

In [30]:
#df[df.iloc[:, 5:-1].gt(0.0, axis=1)]

In [14]:
allIsos.loc[allIsos['eu154'].gt(0) & allIsos['cs137'].gt(0)]

,Burnup,CoolingTime,Enrichment,OrigenReactor,ReactorType,ag109,ag110m,am,am241,am242,...,tc99,u,u232,u233,u234,u235,u236,u238,> zero,== zero
DOD-1|Y013|UO2-b2|DU1,55490.0,0.0,4.94,Dodewaard-1_BWR,bwr,0.04504,0.0,0.3789,0.2053,0.0,...,0.9824,928.0,0.0,0.0,0.14070,5.7630,7.113,914.4,66,19
FDN-1|2F1ZN2|C3|UT,38150.0,0.0,4.90,Fukushima-Daini-1_BWR,bwr,0.04656,0.0,0.0000,0.0000,0.0,...,0.6859,0.0,0.0,0.0,0.32080,17.0500,6.160,967.2,43,42
FDN-1|2F1ZN3|A9|UM,64180.0,0.0,2.10,Fukushima-Daini-1_BWR,bwr,0.17270,0.0,0.0000,0.0000,0.0,...,0.9436,0.0,0.0,0.0,0.06682,0.2247,2.861,986.1,43,42
FDN-1|2F1ZN3|C3|UM,68420.0,0.0,4.90,Fukushima-Daini-1_BWR,bwr,0.11380,0.0,0.0000,0.0000,0.0,...,1.1590,0.0,0.0,0.0,0.22720,3.6010,7.739,977.6,43,42
FDN-1|2F1ZN2|C2|GdT,27890.0,0.0,3.00,Fukushima-Daini-1_BWR,bwr,0.04639,0.0,0.0000,0.0000,0.0,...,0.5104,0.0,0.0,0.0,0.16560,10.9900,3.733,976.0,43,42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VAN-2|EF05|WZtR160|WZtR160-800,73000.0,0.0,4.50,Vandellos-2_PWR,pwr,0.00000,0.0,0.0000,0.6320,0.0,...,1.5900,0.0,0.0,0.0,0.19500,3.8600,7.130,0.0,50,35
VAN-2|EF05|WZR0058|E58-793,74000.0,0.0,4.50,Vandellos-2_PWR,pwr,0.00000,0.0,0.0000,0.2970,0.0,...,1.1200,0.0,0.0,0.0,0.14600,3.0300,6.650,0.0,48,37
VAN-2|EF05|WZR0058|E58-796,74000.0,0.0,4.50,Vandellos-2_PWR,pwr,0.00000,0.0,0.0000,0.3170,0.0,...,0.0000,0.0,0.0,0.0,0.14800,3.1100,6.530,0.0,46,39
VAN-2|EF05|WZR0058|E58-773,74570.0,0.0,4.50,Vandellos-2_PWR,pwr,0.00000,0.0,0.0000,0.3500,0.0,...,0.0000,0.0,0.0,0.0,0.13200,3.1200,6.280,0.0,46,39


### Get new iso list?

In [41]:
pd.set_option("display.max_rows", None, "display.max_columns", None)
col_count = allIsos[allIsos.columns[~allIsos.columns.isin(lbls)]].astype(bool).sum(axis=0)
cols = col_count[col_count.gt(100)]

In [44]:
nucs = cols.index.tolist()[0:-2]
nucs

['am241',
 'am242m',
 'am243',
 'cm242',
 'cm244',
 'cs134',
 'cs137',
 'eu154',
 'nd143',
 'nd144',
 'nd145',
 'nd146',
 'nd148',
 'nd150',
 'np237',
 'pu238',
 'pu239',
 'pu240',
 'pu241',
 'pu242',
 'sm147',
 'sm149',
 'sm150',
 'sm151',
 'sm152',
 'u234',
 'u235',
 'u236',
 'u238']

In [48]:
len(nucs)

29